In [1]:
# encode_and_retrieve_topk.py
# -*- coding: utf-8 -*-
import json, faiss, numpy as np, torch, random
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from peft import PeftModel
import torch.nn.functional as F

# ========= 路径配置 =========
# 已建好的向量库（由 casebank_*_with_embeddings.json 构建）
FAISS_INDEX_PATH = "casebank_A_qwen_cls_cosine.index"
ID_MAP_PATH      = "casebank_A_qwen_cls_idmap.json"

# 待向量化并检索的测试集
TEST_INPUT          = "picked_balanced_around30.json"
TEST_WITH_EMB_OUT   = "A_test_with_embeddings.json"
TOPK_RESULTS_OUT    = "A_test_top4to6_final1.json"   # 改个名字以示区分

# Qwen + LoRA（与建库时完全一致）
BASE_MODEL  = "Qwen/Qwen2.5-1.5B-Instruct"
ADAPTER_DIR = "mbti_lora_qwen1.5b-split_kaggle_ckpt"

MAX_LEN = 512
device = "cuda" if torch.cuda.is_available() else "cpu"

# （可选）设定随机种子，保证复现；不需要可注释掉
# random.seed(42)

# ========= 加载索引 =========
index = faiss.read_index(FAISS_INDEX_PATH)
id_map = json.load(open(ID_MAP_PATH, "r", encoding="utf-8"))

# ========= 加载模型（与建库时一致）=========
tok = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
tok.padding_side = "right"

cfg = AutoConfig.from_pretrained(BASE_MODEL, trust_remote_code=True)
cfg.num_labels = 16

base = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL, config=cfg, trust_remote_code=True,
    device_map={"": device}, low_cpu_mem_usage=True
)
model = PeftModel.from_pretrained(base, ADAPTER_DIR, is_trainable=False).eval().to(device)

# ========= 工具函数 =========
def pick_text(item: dict) -> str:
    """按优先级选一个可用于编码的文本字段。按你的数据结构需要可调整顺序。"""
    for key in ["posts_cleaned_for_embedding", "embed_text", "post_casebank",
                "posts_cleaned", "posts", "text", "query_text"]:
        v = item.get(key)
        if isinstance(v, str) and v.strip():
            return v
    return ""

@torch.no_grad()
def encode_text_to_vec(text: str) -> np.ndarray:
    """
    用 masked mean pooling 提句向量（整段文本表征）并做 L2 归一化，
    与索引端 IndexFlatIP（内积≈余弦）一致。
    """
    ins = tok(text, return_tensors="pt", truncation=True, padding=True, max_length=MAX_LEN).to(device)
    out = model.base_model.model(**ins, output_hidden_states=True, use_cache=False, return_dict=True)
    last_hidden = out.hidden_states[-1]                 # (B, T, H)
    mask = ins["attention_mask"].unsqueeze(-1).float()  # (B, T, 1)

    masked = last_hidden * mask                         # zero out pads
    sum_hidden = masked.sum(dim=1)                      # (B, H)
    lengths = mask.sum(dim=1).clamp(min=1.0)            # (B, 1)
    vec = sum_hidden / lengths                          # (B, H)

    vec = F.normalize(vec, p=2, dim=1)                  # L2 归一化
    return vec.squeeze(0).float().cpu().numpy()         # (H,)

def search_topk(vec: np.ndarray, k: int):
    D, I = index.search(vec.reshape(1, -1), k)
    return [{"score": float(s), **id_map[str(int(i))]} for s, i in zip(D[0], I[0])]

def filter_self_hits(query_text: str, hits: list, k: int):
    """可选：过滤与查询文本完全相同的返回，避免“命中自己”"""
    def norm(s): return " ".join((s or "").lower().split())
    qt = norm(query_text)
    keep = [h for h in hits if norm(h.get("post_casebank", "")) != qt]
    return keep[:k]

# ========= 读取测试集、编码、检索 =========
test = json.load(open(TEST_INPUT, "r", encoding="utf-8"))

with_emb, results = [], []
for it in tqdm(test, desc="Encode & retrieve"):
    text = pick_text(it)
    if not text:
        # 没有可用文本就记空
        it_out = dict(it); it_out["embedding"] = None
        with_emb.append(it_out)
        results.append({"type": it.get("type",""), "query_text": "", "topk_cases": []})
        continue

    emb = encode_text_to_vec(text)              # (H,)
    it_out = dict(it); it_out["embedding"] = emb.tolist()
    with_emb.append(it_out)

    # —— 这里随机决定 k = 4 / 5 / 6，并取前 k 个最相似 —— #
    k_dynamic = random.randint(4, 6)
    topk = search_topk(emb, k=k_dynamic)
    topk = filter_self_hits(text, topk, k_dynamic)   # 不担心命中自己可注释掉

    results.append({
        "type": it.get("type",""),
        "query_text": text,
        "topk_cases": topk
    })

# ========= 落盘 =========
json.dump(with_emb, open(TEST_WITH_EMB_OUT, "w", encoding="utf-8"), ensure_ascii=False, indent=2)
json.dump(results, open(TOPK_RESULTS_OUT, "w", encoding="utf-8"), ensure_ascii=False, indent=2)

print("✅ 向量化完成：", TEST_WITH_EMB_OUT)
print("✅ TopK 随机 4~6（取前 k 个）完成：", TOPK_RESULTS_OUT)


/home/hli962/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-1.5B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Encode & retrieve: 100%|██████████| 325/325 [00:18<00:00, 17.37it/s]


✅ 向量化完成： A_test_with_embeddings.json
✅ TopK 随机 4~6（取前 k 个）完成： A_test_top4to6_final1.json


In [3]:
# encode_and_retrieve_random_proto_global.py
# -*- coding: utf-8 -*-
import json, faiss, numpy as np, torch, random
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from peft import PeftModel
import torch.nn.functional as F

# ========= 路径配置 =========
FAISS_INDEX_PATH = "casebank_A_qwen_cls_cosine.index"   # 还是需要加载一下（embedding一致）
ID_MAP_PATH      = "casebank_A_qwen_cls_idmap.json"     # casebank 映射表

TEST_INPUT          = "picked_balanced_around30.json"
TEST_WITH_EMB_OUT   = "A_test_with_embeddings.json"
TOPK_RESULTS_OUT    = "A_test_random_global3_final1.json"   # Random-proto (全局随机 3)

BASE_MODEL  = "Qwen/Qwen2.5-1.5B-Instruct"
ADAPTER_DIR = "mbti_lora_qwen1.5b-split_kaggle_ckpt"

MAX_LEN = 512
device = "cuda" if torch.cuda.is_available() else "cpu"

# 可选：设置随机种子保证复现
# random.seed(42)

# ========= 加载 casebank =========
id_map = json.load(open(ID_MAP_PATH, "r", encoding="utf-8"))
casebank = list(id_map.values())   # 取出所有样本，后面直接随机抽

# ========= 加载模型 =========
tok = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
tok.padding_side = "right"

cfg = AutoConfig.from_pretrained(BASE_MODEL, trust_remote_code=True)
cfg.num_labels = 16

base = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL, config=cfg, trust_remote_code=True,
    device_map={"": device}, low_cpu_mem_usage=True
)
model = PeftModel.from_pretrained(base, ADAPTER_DIR, is_trainable=False).eval().to(device)

# ========= 工具函数 =========
def pick_text(item: dict) -> str:
    for key in ["posts_cleaned_for_embedding", "embed_text", "post_casebank",
                "posts_cleaned", "posts", "text", "query_text"]:
        v = item.get(key)
        if isinstance(v, str) and v.strip():
            return v
    return ""

@torch.no_grad()
def encode_text_to_vec(text: str) -> np.ndarray:
    ins = tok(text, return_tensors="pt", truncation=True, padding=True, max_length=MAX_LEN).to(device)
    out = model.base_model.model(**ins, output_hidden_states=True, use_cache=False, return_dict=True)
    last_hidden = out.hidden_states[-1]
    mask = ins["attention_mask"].unsqueeze(-1).float()

    masked = last_hidden * mask
    sum_hidden = masked.sum(dim=1)
    lengths = mask.sum(dim=1).clamp(min=1.0)
    vec = sum_hidden / lengths

    vec = F.normalize(vec, p=2, dim=1)
    return vec.squeeze(0).float().cpu().numpy()

# ========= 主流程 =========
test = json.load(open(TEST_INPUT, "r", encoding="utf-8"))

with_emb, results = [], []
for it in tqdm(test, desc="Encode & retrieve (Random-proto GLOBAL)"):
    text = pick_text(it)
    if not text:
        it_out = dict(it); it_out["embedding"] = None
        with_emb.append(it_out)
        results.append({"type": it.get("type",""), "query_text": "", "topk_cases": []})
        continue

    emb = encode_text_to_vec(text)
    it_out = dict(it); it_out["embedding"] = emb.tolist()
    with_emb.append(it_out)

    # === 全局随机抽 3 个 case ===
    topk = random.sample(casebank, 3) if len(casebank) >= 3 else casebank

    results.append({
        "type": it.get("type",""),
        "query_text": text,
        "topk_cases": topk
    })

# ========= 落盘 =========
json.dump(with_emb, open(TEST_WITH_EMB_OUT, "w", encoding="utf-8"), ensure_ascii=False, indent=2)
json.dump(results, open(TOPK_RESULTS_OUT, "w", encoding="utf-8"), ensure_ascii=False, indent=2)

print("✅ 向量化完成：", TEST_WITH_EMB_OUT)
print("✅ Random-proto (全局随机抽 3 个) 完成：", TOPK_RESULTS_OUT)


Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-1.5B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Encode & retrieve (Random-proto GLOBAL): 100%|██████████| 325/325 [00:16<00:00, 19.19it/s]


✅ 向量化完成： A_test_with_embeddings.json
✅ Random-proto (全局随机抽 3 个) 完成： A_test_random_global3_final1.json


In [2]:
# build_casebank_index_qwen_base.py
# -*- coding: utf-8 -*-
import json, faiss, numpy as np, torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

# ======= 路径配置（按你本地修改） =======
ID_MAP_IN          = "casebank_A_qwen_cls_idmap.json"   # 现有的 casebank 元数据（只读）
OUT_INDEX_PATH     = "casebank_qwenbase_cosine.index"   # 新索引（Qwen 基座向量）
OUT_ID_MAP_COPY    = "casebank_qwenbase_idmap.json"     # 复制一份 id_map（保持一致）

BASE_MODEL  = "Qwen/Qwen2.5-1.5B-Instruct"              # 纯基座，不加载 LoRA
MAX_LEN     = 512
BATCH_SIZE  = 64

device = "cuda" if torch.cuda.is_available() else "cpu"

# ======= 加载 Qwen 基座 =======
tok = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
tok.padding_side = "right"

model = AutoModel.from_pretrained(
    BASE_MODEL, trust_remote_code=True,
    torch_dtype=torch.float16 if torch.cuda.is_available() else None
).eval().to(device)

@torch.no_grad()
def embed_texts(texts):
    ins = tok(texts, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LEN).to(device)
    out = model(**ins, output_hidden_states=True, return_dict=True)
    last = out.last_hidden_state                    # (B,T,H)
    mask = ins["attention_mask"].unsqueeze(-1).float()
    vec = (last * mask).sum(1) / mask.sum(1).clamp(min=1e-6)   # mean pooling
    vec = F.normalize(vec, p=2, dim=1)                          # L2 归一化 → 余弦
    return vec.detach().cpu().numpy().astype("float32")         # (B,H)

# ======= 读取 casebank 并编码 =======
id_map = json.load(open(ID_MAP_IN, "r", encoding="utf-8"))
# 保持顺序稳定
items = [id_map[k] for k in sorted(id_map.keys(), key=lambda x: int(x))]

vecs = []
for i in tqdm(range(0, len(items), BATCH_SIZE), desc="Encode casebank with Qwen-base"):
    texts = [ (it.get("post_casebank") or it.get("post") or "") for it in items[i:i+BATCH_SIZE] ]
    vecs.append(embed_texts(texts))
vecs = np.vstack(vecs)  # (N,H), float32

# ======= 建 Faiss 索引（cosine = L2 + IP） =======
index = faiss.IndexFlatIP(vecs.shape[1])
index.add(vecs)
faiss.write_index(index, OUT_INDEX_PATH)

# 复制一份 id_map 以配套新索引
json.dump(id_map, open(OUT_ID_MAP_COPY, "w", encoding="utf-8"), ensure_ascii=False, indent=2)

print(f"✅ 索引已写入: {OUT_INDEX_PATH}")
print(f"✅ id_map 已写入: {OUT_ID_MAP_COPY}")
print(f"📦 向量数: {vecs.shape[0]}, 维度: {vecs.shape[1]}")


Encode casebank with Qwen-base: 100%|██████████| 426/426 [02:39<00:00,  2.67it/s]


✅ 索引已写入: casebank_qwenbase_cosine.index
✅ id_map 已写入: casebank_qwenbase_idmap.json
📦 向量数: 27224, 维度: 1536


In [4]:
# encode_and_retrieve_topk_qwen_base.py
# -*- coding: utf-8 -*-
import json, faiss, numpy as np, torch, random
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

# ========= 路径配置 =========
FAISS_INDEX_PATH   = "casebank_qwenbase_cosine.index"   # 用①生成的新索引
ID_MAP_PATH        = "casebank_qwenbase_idmap.json"     # 与新索引配套的 id_map

TEST_INPUT         = "picked_balanced_around30.json"
TEST_WITH_EMB_OUT  = "QwenBase_test_with_embeddings.json"
TOPK_RESULTS_OUT   = "QwenBase_test_topk3_final1.json"         # e.g., top-3
K                  = 3                                  # 这里配你要的 k（baseline=3）

# 若要 Random-proto（全库随机 3 个），把 MODE 改为 "random_global"
# 若要随机 4–6（取前 k 个最相似），把 MODE 改为 "topk_4to6"
MODE               = "topk"  # "topk" | "topk_4to6" | "random_global"

BASE_MODEL  = "Qwen/Qwen2.5-1.5B-Instruct"
MAX_LEN     = 512
device = "cuda" if torch.cuda.is_available() else "cpu"

# ========= 加载索引 & id_map =========
index = faiss.read_index(FAISS_INDEX_PATH)
id_map = json.load(open(ID_MAP_PATH, "r", encoding="utf-8"))
casebank_all = list(id_map.values())  # 给 random_global 用

# ========= 加载 Qwen 基座 =========
tok = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token
tok.padding_side = "right"

model = AutoModel.from_pretrained(
    BASE_MODEL, trust_remote_code=True,
    torch_dtype=torch.float16 if torch.cuda.is_available() else None
).eval().to(device)

@torch.no_grad()
def encode_text_to_vec(text: str) -> np.ndarray:
    ins = tok(text, return_tensors="pt", truncation=True, padding=True, max_length=MAX_LEN).to(device)
    out = model(**ins, output_hidden_states=True, return_dict=True)
    last = out.last_hidden_state
    mask = ins["attention_mask"].unsqueeze(-1).float()
    vec = (last * mask).sum(1) / mask.sum(1).clamp(min=1e-6)
    vec = F.normalize(vec, p=2, dim=1)
    return vec.squeeze(0).detach().cpu().numpy().astype("float32")

def search_topk(vec: np.ndarray, k: int):
    D, I = index.search(vec.reshape(1, -1), k)
    return [{"score": float(s), **id_map[str(int(i))]} for s, i in zip(D[0], I[0])]

def filter_self_hits(query_text: str, hits: list, k: int = None):
    def norm(s): return " ".join((s or "").lower().split())
    qt = norm(query_text)
    keep = [h for h in hits if norm(h.get("post_casebank", "")) != qt]
    return keep if k is None else keep[:k]

# ========= 读取测试集、编码、检索 =========
test = json.load(open(TEST_INPUT, "r", encoding="utf-8"))

with_emb, results = [], []
desc = {
    "topk": "Top-k (Qwen-base)",
    "topk_4to6": "Top-k random 4–6 (Qwen-base)",
    "random_global": "Random-proto global 3 (Qwen-base)"
}[MODE]

for it in tqdm(test, desc=f"Encode & retrieve · {desc}"):
    # 取文本
    text = None
    for key in ["posts_cleaned_for_embedding", "embed_text", "post_casebank",
                "posts_cleaned", "posts", "text", "query_text"]:
        v = it.get(key)
        if isinstance(v, str) and v.strip():
            text = v
            break
    if not text:
        it_out = dict(it); it_out["embedding"] = None
        with_emb.append(it_out)
        results.append({"type": it.get("type",""), "query_text": "", "topk_cases": []})
        continue

    emb = encode_text_to_vec(text)
    it_out = dict(it); it_out["embedding"] = emb.tolist()
    with_emb.append(it_out)

    # 三种模式
    if MODE == "topk":
        topk = filter_self_hits(text, search_topk(emb, k=K), K)

    elif MODE == "topk_4to6":
        k_dynamic = random.randint(4, 6)
        topk = filter_self_hits(text, search_topk(emb, k=k_dynamic), k_dynamic)

    elif MODE == "random_global":
        # 全库随机 3 个（与你之前的 Random-proto 需求一致）
        pool = casebank_all
        topk = random.sample(pool, 3) if len(pool) >= 3 else pool

    else:
        raise ValueError(f"Unknown MODE: {MODE}")

    results.append({
        "type": it.get("type",""),
        "query_text": text,
        "topk_cases": topk
    })

# ========= 落盘 =========
json.dump(with_emb, open(TEST_WITH_EMB_OUT, "w", encoding="utf-8"), ensure_ascii=False, indent=2)
json.dump(results, open(TOPK_RESULTS_OUT, "w", encoding="utf-8"), ensure_ascii=False, indent=2)

print("✅ 向量化完成：", TEST_WITH_EMB_OUT)
print(f"✅ 检索完成（{desc}）: {TOPK_RESULTS_OUT}")


Encode & retrieve · Top-k (Qwen-base): 100%|██████████| 325/325 [00:06<00:00, 49.58it/s]


✅ 向量化完成： QwenBase_test_with_embeddings.json
✅ 检索完成（Top-k (Qwen-base)）: QwenBase_test_topk3_final1.json
